Information about the data set

* last updated at March 18, 2023

In [58]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import  pandas as pd 

1. Loading the data set in a data frame

In [59]:
ev_pop = pd.read_csv('data/Electric_Vehicle_Population_Data.csv')
ev_pop.head(5)

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,5YJ3E1EB2J,Suffolk,Suffolk,VA,23435.0,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0,NaN,476647986,POINT (-76.42443 36.8752),NaN,5.180008e+10
1,5YJ3E1ECXL,Yakima,Yakima,WA,98908.0,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,308,0,14.0,103490145,POINT (-120.56916 46.58514),PACIFICORP,5.307700e+10
2,WA1LAAGE7M,Yakima,Yakima,WA,98908.0,2021,AUDI,E-TRON,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,222,0,14.0,144941534,POINT (-120.56916 46.58514),PACIFICORP,5.307700e+10
3,5YJ3E1EA1K,Danville,Danville,VA,24541.0,2019,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,220,0,NaN,168513922,POINT (-79.4172 36.58598),NaN,5.159000e+10
4,1FADP5CU9E,Norfolk,Norfolk,VA,23518.0,2014,FORD,C-MAX,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,19,0,NaN,150749378,POINT (-76.21549 36.92478),NaN,5.171001e+10


In [60]:
ev_pop.columns

Index(['VIN (1-10)', 'County', 'City', 'State', 'Postal Code', 'Model Year',
       'Make', 'Model', 'Electric Vehicle Type',
       'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'Electric Range',
       'Base MSRP', 'Legislative District', 'DOL Vehicle ID',
       'Vehicle Location', 'Electric Utility', '2020 Census Tract'],
      dtype='object')

2. extracting the information that we need 

In [61]:
df_ev_pop = ev_pop[['DOL Vehicle ID', 'Make','Model','Vehicle Location', 'County', 'City', 'State', 'County', 'Model Year']]

This dataset needs time at least we need to know when it is records etc 

In [62]:
df_ev_pop["last_updated_at"]= pd.to_datetime("03/18/2023")
df_ev_pop.head()

,DOL Vehicle ID,Make,Model,Vehicle Location,County,City,State,County,Model Year,last_updated_at
0,476647986,TESLA,MODEL 3,POINT (-76.42443 36.8752),Suffolk,Suffolk,VA,Suffolk,2018,2023-03-18
1,103490145,TESLA,MODEL 3,POINT (-120.56916 46.58514),Yakima,Yakima,WA,Yakima,2020,2023-03-18
2,144941534,AUDI,E-TRON,POINT (-120.56916 46.58514),Yakima,Yakima,WA,Yakima,2021,2023-03-18
3,168513922,TESLA,MODEL 3,POINT (-79.4172 36.58598),Danville,Danville,VA,Danville,2019,2023-03-18
4,150749378,FORD,C-MAX,POINT (-76.21549 36.92478),Norfolk,Norfolk,VA,Norfolk,2014,2023-03-18


In [63]:
df_ev_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121978 entries, 0 to 121977
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   DOL Vehicle ID    121978 non-null  int64         
 1   Make              121978 non-null  object        
 2   Model             121874 non-null  object        
 3   Vehicle Location  121946 non-null  object        
 4   County            121976 non-null  object        
 5   City              121976 non-null  object        
 6   State             121978 non-null  object        
 7   County            121976 non-null  object        
 8   Model Year        121978 non-null  int64         
 9   last_updated_at   121978 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 9.3+ MB


we need to split Vehicle Location column in to two lat and long 

In [64]:
df_ev_pop[["val", "Latitude", "Longitude"]] = df_ev_pop['Vehicle Location'].str.split(' ', expand=True)
# 

# df_ev_pop.drop(columns=['val'])
# removing paranthesis 
df_ev_pop["Latitude"] =  df_ev_pop["Latitude"].str[1:]
df_ev_pop["Longitude"] =df_ev_pop["Longitude"].str.replace(')',"")
df_ev_pop["Longitude"]= df_ev_pop["Longitude"].str.strip()
df_ev_pop["Latitude"] =  df_ev_pop["Latitude"].str.strip()
df_ev_pop.head()

,DOL Vehicle ID,Make,Model,Vehicle Location,County,City,State,County,Model Year,last_updated_at,val,Latitude,Longitude
0,476647986,TESLA,MODEL 3,POINT (-76.42443 36.8752),Suffolk,Suffolk,VA,Suffolk,2018,2023-03-18,POINT,-76.42443,36.8752
1,103490145,TESLA,MODEL 3,POINT (-120.56916 46.58514),Yakima,Yakima,WA,Yakima,2020,2023-03-18,POINT,-120.56916,46.58514
2,144941534,AUDI,E-TRON,POINT (-120.56916 46.58514),Yakima,Yakima,WA,Yakima,2021,2023-03-18,POINT,-120.56916,46.58514
3,168513922,TESLA,MODEL 3,POINT (-79.4172 36.58598),Danville,Danville,VA,Danville,2019,2023-03-18,POINT,-79.4172,36.58598
4,150749378,FORD,C-MAX,POINT (-76.21549 36.92478),Norfolk,Norfolk,VA,Norfolk,2014,2023-03-18,POINT,-76.21549,36.92478


In [65]:
df_ev_pop =df_ev_pop.drop(columns=['val'])

In [66]:
df_ev_pop["quantity"]= int(1)

In [67]:
df_ev_pop.columns 

Index(['DOL Vehicle ID', 'Make', 'Model', 'Vehicle Location', 'County', 'City',
       'State', 'County', 'Model Year', 'last_updated_at', 'Latitude',
       'Longitude', 'quantity'],
      dtype='object')

In [68]:
df_ev_pop.rename(columns={'DOL Vehicle ID': 'dv_pop_DOL_Vehicle ID', 
                          'Make': 'dv_pop_Make',
                          'Model': 'dv_pop_Model',
                          'Vehicle Location': 'dv_pop_Vehicle_Location',
                          'Model Year': 'dv_pop_Model_Year',
                          'last_updated_at': 'dv_pop_last_updated_at',
                          'quantity': 'dv_pop_quantity',
                          }, inplace=True)

In [69]:
df_ev_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121978 entries, 0 to 121977
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   dv_pop_DOL_Vehicle ID    121978 non-null  int64         
 1   dv_pop_Make              121978 non-null  object        
 2   dv_pop_Model             121874 non-null  object        
 3   dv_pop_Vehicle_Location  121946 non-null  object        
 4   County                   121976 non-null  object        
 5   City                     121976 non-null  object        
 6   State                    121978 non-null  object        
 7   County                   121976 non-null  object        
 8   dv_pop_Model_Year        121978 non-null  int64         
 9   dv_pop_last_updated_at   121978 non-null  datetime64[ns]
 10  Latitude                 121946 non-null  object        
 11  Longitude                121946 non-null  object        
 12  dv_pop_quantity 

In [72]:
len(df_ev_pop.City.unique())

648

In [75]:
from geopy.geocoders import Nominatim
import math
geolocator = Nominatim(user_agent="my_app")
import math 
city_list = df_ev_pop.City.unique().tolist()
city_lat = []
city_long = []
i =0
for city in  city_list:
    i+=1
    location = geolocator.geocode(city + ", " + 'WA')
    if location is None or math.isnan(location.latitude) or math.isnan(location.longitude):
        city_lat.append(None)
        city_lat.append("null")
        city_long.append("null") 
    else:
        city_lat.append(location.latitude)
        city_long.append(location.longitude)
    print("done")



269 done
270 done
271 done
272 done
273 done
274 done


TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [ ]:
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="my_app")

# city_lat = []
# city_long = []

# for index, row in df_ev_pop.iterrows():
#     location = geolocator.geocode(row["dv_pop_City"] + ", " + row["dv_pop_State"])
#     city_lat.append(location.latitude)
#     city_long.append(location.longitude)
    
# df_ev_pop[city_lat] = city_lat
# df_ev_pop[city_long] = city_long

we can drop Vehicle Location 

-----------------------

2. ev_station data set 

In [14]:
# pd.set_option("display.max_columns",10 )
ev_station = pd.read_csv('data/alt_fuel_stations.csv')
ev_station.head(5)

,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Plus4,Station Phone,Status Code,...,EV Pricing (French),LPG Nozzle Types,Hydrogen Pressures,Hydrogen Standards,CNG Fill Type Code,CNG PSI,CNG Vehicle Class,LNG Vehicle Class,EV On-Site Renewable Source,Restricted Access
0,CNG,Spire - Montgomery Operations Center,2951 Chestnut St,NaN,Montgomery,AL,36107,NaN,NaN,E,...,NaN,NaN,NaN,NaN,B,3600,MD,NaN,NaN,NaN
1,CNG,PS Energy - Atlanta,340 Whitehall St,"From I-7585 N, exit 91 to Central Ave, left on...",Atlanta,GA,30303,NaN,770-350-3000,E,...,NaN,NaN,NaN,NaN,Q,3600,MD,NaN,NaN,NaN
2,CNG,Metropolitan Atlanta Rapid Transit Authority,2424 Piedmont Rd NE,NaN,Atlanta,GA,30324,NaN,NaN,E,...,NaN,NaN,NaN,NaN,Q,3000,LD,NaN,NaN,NaN
3,CNG,United Parcel Service,270 Marvin Miller Dr,NaN,Atlanta,GA,30336,NaN,NaN,E,...,NaN,NaN,NaN,NaN,B,3600,HD,NaN,NaN,NaN
4,CNG,Clean Energy - Texas Department of Transportation,7721A Washington St,"I-10, Washington Ave exit, 1.5 blocks to the s...",Houston,TX,77007,NaN,866-809-4869,E,...,NaN,NaN,NaN,NaN,Q,3000 3600,MD,NaN,NaN,NaN


In [15]:
ev_station.columns

Index(['Fuel Type Code', 'Station Name', 'Street Address',
       'Intersection Directions', 'City', 'State', 'ZIP', 'Plus4',
       'Station Phone', 'Status Code', 'Expected Date',
       'Groups With Access Code', 'Access Days Time', 'Cards Accepted',
       'BD Blends', 'NG Fill Type Code', 'NG PSI', 'EV Level1 EVSE Num',
       'EV Level2 EVSE Num', 'EV DC Fast Count', 'EV Other Info', 'EV Network',
       'EV Network Web', 'Geocode Status', 'Latitude', 'Longitude',
       'Date Last Confirmed', 'ID', 'Updated At', 'Owner Type Code',
       'Federal Agency ID', 'Federal Agency Name', 'Open Date',
       'Hydrogen Status Link', 'NG Vehicle Class', 'LPG Primary',
       'E85 Blender Pump', 'EV Connector Types', 'Country',
       'Intersection Directions (French)', 'Access Days Time (French)',
       'BD Blends (French)', 'Groups With Access Code (French)',
       'Hydrogen Is Retail', 'Access Code', 'Access Detail Code',
       'Federal Agency Code', 'Facility Type', 'CNG Dispenser N

In [16]:
ev_station_v1 = ev_station[['ID','Updated At', 'Status Code', "Station Name", 'Fuel Type Code', "Access Code", "Street Address", "City", "State","Country", "ZIP", "Latitude", "Longitude", "Open Date", 'EV Pricing'] ]


In [17]:
len(ev_station_v1)

56800

In [18]:
ev_station_v1.head()

,ID,Updated At,Status Code,Station Name,Fuel Type Code,Access Code,Street Address,City,State,Country,ZIP,Latitude,Longitude,Open Date,EV Pricing
0,17,2021-07-29 15:08:10 UTC,E,Spire - Montgomery Operations Center,CNG,private,2951 Chestnut St,Montgomery,AL,US,36107,32.367916,-86.267021,2010-12-01,NaN
1,42,2021-07-26 13:24:55 UTC,E,PS Energy - Atlanta,CNG,public,340 Whitehall St,Atlanta,GA,US,30303,33.745843,-84.398837,1994-07-15,NaN
2,45,2021-07-27 21:03:31 UTC,E,Metropolitan Atlanta Rapid Transit Authority,CNG,private,2424 Piedmont Rd NE,Atlanta,GA,US,30324,33.821911,-84.367461,1996-12-15,NaN
3,64,2021-04-08 20:32:57 UTC,E,United Parcel Service,CNG,private,270 Marvin Miller Dr,Atlanta,GA,US,30336,33.760256,-84.543822,1997-01-01,NaN
4,72,2021-07-20 14:30:28 UTC,E,Clean Energy - Texas Department of Transportation,CNG,public,7721A Washington St,Houston,TX,US,77007,29.779918,-95.435385,1996-12-15,NaN


fuel type cng = Compressed Natural Gas (CNG) is a gasoline and diesel fuel alternative consisting primarily of methane. The gas is associated with other fossil fuels (coal or oil) and is created by methanogenic organisms in landfill

E85= E85 (or flex fuel) is a term that refers to high-level ethanol-gasoline blends containing 51% to 83% ethanol, depending on geography and season (see Fuel Properties and E85 Flex Fuel Specification).

ELEC : electric 

LPG= LNG is natural gas in its liquid form. 

BD: ?

HY : Hydrogen Fueling

In [19]:
ev_station_v1["Fuel Type Code"].unique()

array(['CNG', 'E85', 'ELEC', 'LNG', 'LPG', 'BD', 'HY'], dtype=object)

- at the moment we only need WA
- since we are interested in ev chars we do not need other charging stations information, we filter them out 
- we need to also bring the charging stations status
- status code :  e  is available , t is temprorarly closed , we can change this information with active and temprorily closed 
- we can consider this data set as 

In [20]:
ev_station_v2 = ev_station_v1.loc[ev_station_v1["Fuel Type Code"]=='ELEC']
ev_station_v3 = ev_station_v2.loc[ev_station_v1["State"]=='WA']

In [21]:
len(ev_station_v3)

1686

In [22]:
ev_station_v3.head()

,ID,Updated At,Status Code,Station Name,Fuel Type Code,Access Code,Street Address,City,State,Country,ZIP,Latitude,Longitude,Open Date,EV Pricing
2348,33351,2021-03-11 23:22:17 UTC,E,City of Lacey - City Hall Parking,ELEC,public,420 College St,Lacey,WA,US,98503,47.044011,-122.822404,2018-01-15,NaN
2431,33717,2021-03-11 23:22:17 UTC,E,Seattle-Tacoma International Airport - General...,ELEC,public,17801 Pacific Hwy S,Seattle,WA,US,98188,47.443377,-122.296229,2010-03-01,Free
2880,35337,2021-03-11 23:22:17 UTC,E,Haggen Food & Pharmacy,ELEC,public,1313 Cooper Point Rd SW,Olympia,WA,US,98502,47.035421,-122.938962,2009-08-15,Free
2975,35620,2021-03-11 23:22:17 UTC,E,Avista Corp,ELEC,public,1411 E Mission Ave,Spokane,WA,US,99252,47.673347,-117.388933,2010-04-15,Free
2976,35621,2021-03-11 23:22:17 UTC,E,Steam Plant Grill,ELEC,public,159 S Lincoln St,Spokane,WA,US,99201,47.654998,-117.425025,2010-04-15,Free


checking to see if we need any transformation on the pricing info. it seems we need an big transformation. 

In [23]:
ev_station_v3["quantity"]= int(1)

In [24]:
ev_station_v3.columns

Index(['ID', 'Updated At', 'Status Code', 'Station Name', 'Fuel Type Code',
       'Access Code', 'Street Address', 'City', 'State', 'Country', 'ZIP',
       'Latitude', 'Longitude', 'Open Date', 'EV Pricing', 'quantity'],
      dtype='object')

In [25]:
ev_station_v3['Latitude']= ev_station_v3['Latitude'].astype(str)
ev_station_v3['Longitude']= ev_station_v3['Longitude'].astype(str)

In [26]:
# ev_station_v3["EV Pricing"].unique()

------------------- 

3. combining two data frames to one df 

we need to merge two data frame 
we need to check if the sizes are staied the same

In [27]:
print(ev_station_v3.shape)
print(df_ev_pop.shape)

(1686, 16)
(121978, 13)


In [28]:
ev_station_v3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1686 entries, 2348 to 56750
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              1686 non-null   int64 
 1   Updated At      1686 non-null   object
 2   Status Code     1686 non-null   object
 3   Station Name    1686 non-null   object
 4   Fuel Type Code  1686 non-null   object
 5   Access Code     1686 non-null   object
 6   Street Address  1686 non-null   object
 7   City            1686 non-null   object
 8   State           1686 non-null   object
 9   Country         1686 non-null   object
 10  ZIP             1686 non-null   object
 11  Latitude        1686 non-null   object
 12  Longitude       1686 non-null   object
 13  Open Date       1602 non-null   object
 14  EV Pricing      653 non-null    object
 15  quantity        1686 non-null   int64 
dtypes: int64(2), object(14)
memory usage: 223.9+ KB


In [29]:
result = pd.merge(ev_station_v3,
                  df_ev_pop,
                  on = ['Latitude', 'Longitude'],
                  how='outer',
                  indicator=True
)

In [30]:
result.head()

,ID,Updated At,Status Code,Station Name,Fuel Type Code,Access Code,Street Address,City,State,Country,...,dv_pop_Model,dv_pop_Vehicle_Location,dv_pop_County,dv_pop_City,dv_pop_State,dv_pop_County,dv_pop_Model_Year,dv_pop_last_updated_at,dv_pop_quantity,_merge
0,33351.0,2021-03-11 23:22:17 UTC,E,City of Lacey - City Hall Parking,ELEC,public,420 College St,Lacey,WA,US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,left_only
1,33717.0,2021-03-11 23:22:17 UTC,E,Seattle-Tacoma International Airport - General...,ELEC,public,17801 Pacific Hwy S,Seattle,WA,US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,left_only
2,35337.0,2021-03-11 23:22:17 UTC,E,Haggen Food & Pharmacy,ELEC,public,1313 Cooper Point Rd SW,Olympia,WA,US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,left_only
3,35620.0,2021-03-11 23:22:17 UTC,E,Avista Corp,ELEC,public,1411 E Mission Ave,Spokane,WA,US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,left_only
4,35621.0,2021-03-11 23:22:17 UTC,E,Steam Plant Grill,ELEC,public,159 S Lincoln St,Spokane,WA,US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,left_only


In [31]:
result.columns

Index(['ID', 'Updated At', 'Status Code', 'Station Name', 'Fuel Type Code',
       'Access Code', 'Street Address', 'City', 'State', 'Country', 'ZIP',
       'Latitude', 'Longitude', 'Open Date', 'EV Pricing', 'quantity',
       'dv_pop_DOL_Vehicle ID', 'dv_pop_Make', 'dv_pop_Model',
       'dv_pop_Vehicle_Location', 'dv_pop_County', 'dv_pop_City',
       'dv_pop_State', 'dv_pop_County', 'dv_pop_Model_Year',
       'dv_pop_last_updated_at', 'dv_pop_quantity', '_merge'],
      dtype='object')

In [32]:
print(ev_station_v3.shape)
print(df_ev_pop.shape)

(1686, 16)
(121978, 13)


In [33]:
result.shape

(123664, 28)

In [34]:
121978 - 123664

-1686

In [36]:
result['_merge'].unique()

['left_only', 'right_only']
Categories (3, object): ['left_only', 'right_only', 'both']

In [40]:
# Concatenate city and state columns with a comma
result["City"][0] + ", " + result["State"][0]

'Lacey, WA'

In [54]:

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_app")

location = geolocator.geocode(result["City"][0] + ", " + result["State"][0])

print(location.latitude, location.longitude)

47.0263876 -122.8072257


In [46]:
len(result.City.unique())

191

In [48]:
result.State.unique()

array(['WA', nan], dtype=object)

In [ ]:
city_lat = []
city_long = []
for index, row in result.iterrows():
    location = geolocator.geocode(row["City"] + ", " + row["State"])
    city_lat.append(location.latitude)
    city_long.append(location.longitude)
    
result[city_lat] = city_lat
result[city_long] = city_long